In [1]:
import math
import random
import re
import matplotlib.pyplot as plt

In [2]:
from jmetal.core.problem import PermutationProblem
from jmetal.core.solution import PermutationSolution

In [3]:
from jmetal.algorithm.singleobjective.genetic_algorithm import GeneticAlgorithm
from jmetal.algorithm.singleobjective.evolution_strategy import EvolutionStrategy
from jmetal.operator import BinaryTournamentSelection,PolynomialMutation
from jmetal.operator.crossover import PMXCrossover
from jmetal.operator.mutation import PermutationSwapMutation
from jmetal.util.comparator import MultiComparator
from jmetal.util.density_estimator import CrowdingDistance
from jmetal.util.ranking import FastNonDominatedRanking
from jmetal.util.termination_criterion import StoppingByEvaluations

In [4]:
class TSP(PermutationProblem):
    """ Class representing TSP Problem. """

    def __init__(self, instance: str = None):
        super(TSP, self).__init__()

        distance_matrix, number_of_cities = self.__read_from_file(instance)

        self.distance_matrix = distance_matrix

        self.obj_directions = [self.MINIMIZE]
        self.number_of_variables = number_of_cities
        self.number_of_objectives = 1
        self.number_of_constraints = 0

    def __read_from_file(self, filename: str):
        """
        This function reads a TSP Problem instance from a file.

        :param filename: File which describes the instance.
        :type filename: str.
        """

        if filename is None:
            raise FileNotFoundError('Filename can not be None')

        with open(filename) as file:
            lines = file.readlines()
            data = [line.lstrip() for line in lines if line != ""]

            dimension = re.compile(r'[^\d]+')

            for item in data:
                if item.startswith('DIMENSION'):
                    dimension = int(dimension.sub('', item))
                    break

            c = [-1.0] * (2 * dimension)

            for item in data:
                if item[0].isdigit():
                    j, city_a, city_b = [x.strip() for x in item.split(' ')]
                    c[2 * (int(j) - 1)] = float(city_a)
                    c[2 * (int(j) - 1) + 1] = float(city_b)

            matrix = [[-1] * dimension for _ in range(dimension)]

            for k in range(dimension):
                matrix[k][k] = 0

                for j in range(k + 1, dimension):
                    dist = math.sqrt((c[k * 2] - c[j * 2]) ** 2 + (c[k * 2 + 1] - c[j * 2 + 1]) ** 2)
                    #dist = round(dist)
                    matrix[k][j] = dist
                    matrix[j][k] = dist

            return matrix, dimension

    def evaluate(self, solution: PermutationSolution) -> PermutationSolution:
        fitness = 0

        for i in range(self.number_of_variables - 1):
            x = solution.variables[i]
            y = solution.variables[i + 1]

            fitness += self.distance_matrix[x][y]

        first_city, last_city = solution.variables[0], solution.variables[-1]
        fitness += self.distance_matrix[first_city][last_city]

        solution.objectives[0] = fitness

        return solution


    def create_solution(self) -> PermutationSolution:
        new_solution = PermutationSolution(number_of_variables=self.number_of_variables,
                                           number_of_objectives=self.number_of_objectives)
        new_solution.variables = random.sample(range(self.number_of_variables), k=self.number_of_variables)

        return new_solution


    @property
    def number_of_cities(self):
        return self.number_of_variables

    def get_name(self):
        return 'Symmetric TSP'


In [5]:
def calc_tsp_ga(file_location,population_size_ga,offspring_population_size_ga,max_evaluation_ga):
    problem = TSP(instance=file_location)

    #print('Cities: ', problem.number_of_variables)

    algorithm = GeneticAlgorithm(
        problem=problem,
        population_size=population_size_ga,
        offspring_population_size=offspring_population_size_ga,
        mutation=PermutationSwapMutation(1.0 / problem.number_of_variables),
        crossover=PMXCrossover(0.8),
        selection=BinaryTournamentSelection(
            MultiComparator([FastNonDominatedRanking.get_comparator(),
                             CrowdingDistance.get_comparator()])),
        termination_criterion=StoppingByEvaluations(max = max_evaluation_ga)
    )

    algorithm.run()
    result = algorithm.get_result()
    
    return (algorithm.get_name(),problem.get_name(),result.variables,result.objectives[0],algorithm.total_computing_time)

In [6]:
djibouti_tsp_dat = '/dj38.tsp'
qatar_tsp_dat = '/qa194.tsp'

In [11]:
test1_3 = calc_tsp_ga(djibouti_tsp_dat,200,150,2500000)

In [12]:
print(test1_3)

('Genetic algorithm', 'Symmetric TSP', [34, 22, 19, 1, 0, 9, 13, 20, 28, 31, 29, 25, 24, 21, 23, 18, 17, 15, 8, 7, 14, 12, 6, 5, 3, 2, 4, 10, 11, 16, 27, 26, 30, 35, 33, 32, 36, 37], 8387.553976264137, 249.5164954662323)


In [13]:
test2_2 = calc_tsp_ga(qatar_tsp_dat,150,150,2500000)

In [14]:
print(test2_2)

('Genetic algorithm', 'Symmetric TSP', [37, 30, 72, 99, 107, 147, 127, 132, 134, 135, 152, 164, 156, 140, 125, 126, 100, 103, 110, 113, 108, 101, 77, 69, 28, 14, 11, 26, 17, 23, 25, 20, 95, 169, 179, 180, 176, 187, 183, 174, 167, 166, 158, 154, 76, 67, 39, 38, 33, 27, 56, 73, 59, 71, 74, 44, 42, 34, 41, 40, 57, 66, 114, 184, 192, 186, 182, 168, 129, 98, 62, 5, 3, 1, 0, 58, 89, 97, 124, 15, 10, 2, 4, 8, 9, 50, 36, 21, 6, 7, 61, 88, 93, 90, 92, 80, 87, 118, 148, 160, 163, 155, 81, 13, 16, 24, 75, 133, 144, 145, 141, 12, 70, 79, 86, 116, 120, 142, 122, 123, 48, 29, 18, 31, 45, 47, 52, 46, 63, 117, 146, 151, 149, 153, 162, 178, 189, 193, 185, 173, 121, 112, 131, 139, 136, 138, 137, 172, 188, 190, 177, 170, 159, 161, 157, 150, 130, 91, 82, 96, 104, 115, 119, 128, 143, 171, 175, 181, 191, 165, 106, 94, 78, 83, 65, 60, 32, 68, 22, 35, 19, 64, 85, 84, 102, 105, 111, 109, 55, 43, 49, 54, 53, 51], 25830.675104952894, 2508.474024772644)
